In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
from sklearn import datasets
from sklearn import metrics as m
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
from itertools import combinations
from sklearn import datasets
from sklearn import metrics as m
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd
from itertools import combinations
from sklearn.svm import SVC
import xgboost as xgb
from thundersvm import SVC as svmgpu
from sklearn.utils import shuffle
import itertools
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import LabelEncoder

randomseed = 42
np.random.seed(randomseed)

<IPython.core.display.Javascript object>

In [3]:
def swapcolumns(trainval, testval, coldindexval):
    trainval[trainval != coldindexval] = 5
    testval[testval != coldindexval] = 5

    trainval[trainval == coldindexval] = 0
    trainval[trainval == 5] = 1

    testval[testval == coldindexval] = 0
    testval[testval == 5] = 1

    return trainval, testval

<IPython.core.display.Javascript object>

In [4]:
# data = pd.read_csv("../dataset/ionosphere.data")
# data = shuffle(data)

# le = LabelEncoder()
# data.iloc[:, -1] = le.fit_transform(data.iloc[:, -1])
# x = data.iloc[:, :-1]
# y = data.iloc[:, -1]

<IPython.core.display.Javascript object>

In [5]:
data = datasets.load_wine()
x = data.data
y = data.target.ravel()
print(np.unique(y))

[0 1 2]


<IPython.core.display.Javascript object>

In [6]:
xtrain, xtest, ytrain_original, ytest_original = train_test_split(
    x, y, test_size=0.3, random_state=10
)

<IPython.core.display.Javascript object>

In [7]:
xgbmodel = xgb.XGBClassifier(random_state=randomseed)
xgbmodel.fit(xtrain, ytrain_original)
print(m.accuracy_score(ytest_original, xgbmodel.predict(xtest)))

0.9444444444444444


<IPython.core.display.Javascript object>

In [8]:
cvscore = cross_val_score(xgbmodel, x, y, cv=10)
print("mean= ", np.mean(cvscore), ", std= ", np.std(cvscore))

mean=  0.9669590643274854 , std=  0.03668512923916929


<IPython.core.display.Javascript object>

Confusion matrix based models
====

In [9]:
clf = []
acc = []
predictedprobablities = []
predictedconfusionmatrix_proba = []
acc_train = []

<IPython.core.display.Javascript object>

In [10]:
# 1. baseline classifiers

<IPython.core.display.Javascript object>

In [11]:
# ===========================
ytrain = ytrain_original.copy()
ytest = ytest_original.copy()
ytrain, ytest = swapcolumns(ytrain, ytest, 0)
# =================================================

rf = RandomForestClassifier(random_state=randomseed, n_estimators=50)
rf.fit(xtrain, ytrain)
rfpred = rf.predict(xtest)
print(m.accuracy_score(ytest, rfpred))

clf.append(rf)
acc.append(m.accuracy_score(ytest, rfpred))
predictedprobablities.append(rf.predict_proba(xtest))

confmat = m.confusion_matrix(ytest, rfpred)
confsumh = np.sum(confmat, axis=0)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
predictedconfusionmatrix_proba.append(propconfmat / 100)

0.9814814814814815


<IPython.core.display.Javascript object>

In [12]:
# =================================================
# classs 1
# =================================================
ytrain = ytrain_original.copy()
ytest = ytest_original.copy()
ytrain, ytest = swapcolumns(ytrain, ytest, 1)
# =================================================

rf = RandomForestClassifier(random_state=randomseed, n_estimators=50)
rf.fit(xtrain, ytrain)
rfpred = rf.predict(xtest)
print(m.accuracy_score(ytest, rfpred))

clf.append(rf)
acc.append(m.accuracy_score(ytest, rfpred))
predictedprobablities.append(rf.predict_proba(xtest))

confmat = m.confusion_matrix(ytest, rfpred)
confsumh = np.sum(confmat, axis=0)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
predictedconfusionmatrix_proba.append(propconfmat / 100)


0.9444444444444444


<IPython.core.display.Javascript object>

In [13]:
#=================================================
# classs 2
#=================================================

ytrain=ytrain_original.copy()
ytest=ytest_original.copy()
ytrain,ytest= swapcolumns(ytrain,ytest,2)

#=================================================

rf=RandomForestClassifier(random_state=randomseed, n_estimators=50)
rf.fit(xtrain,ytrain)
rfpred=rf.predict(xtest)
print(m.accuracy_score(ytest,rfpred))

clf.append(rf)
acc.append(m.accuracy_score(ytest,rfpred))
predictedprobablities.append(rf.predict_proba(xtest))

confmat=m.confusion_matrix(ytest,rfpred)
confsumh=np.sum(confmat,axis=0)
propconfmat=confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[:,i]= 100*propconfmat[:,i]/confsumh[i] 
predictedconfusionmatrix_proba.append(propconfmat/100)


0.9814814814814815


<IPython.core.display.Javascript object>

In [14]:
# pc1_all = []
# pc2_all = []

# for i in range(len(predictedconfusionmatrix_proba)):

#     pc1 = ( predictedprobablities[i][:, 0]    )
#     pc2 = (  predictedprobablities[i][:, 1]    )

#     pc1_all.append(pc1)
#     pc2_all.append(pc2)


<IPython.core.display.Javascript object>

In [15]:

ci0 = ci1 = ci2 = 1 

p_k1_c1 = ci0 * (
    predictedprobablities[0][:, 0] * predictedconfusionmatrix_proba[0][1][0]
    + predictedprobablities[0][:, 1] * predictedconfusionmatrix_proba[0][1][1]
)


p_k2_c1 = ci0 * (
    predictedprobablities[1][:, 0] * predictedconfusionmatrix_proba[1][1][0]
    + predictedprobablities[1][:, 1] * predictedconfusionmatrix_proba[1][1][1]
)

p_k3_c1 = ci0 * (
    predictedprobablities[2][:, 0] * predictedconfusionmatrix_proba[2][1][0]
    + predictedprobablities[2][:, 1] * predictedconfusionmatrix_proba[2][1][1]
)


p_k1_c0 = ci0 * (
    predictedprobablities[0][:, 0] * predictedconfusionmatrix_proba[0][0][0]
    + predictedprobablities[0][:, 1] * predictedconfusionmatrix_proba[0][0][1]
) + (p_k2_c1 * p_k3_c1)


p_k2_c0 = ci0 * (
    predictedprobablities[1][:, 0] * predictedconfusionmatrix_proba[1][0][0]
    + predictedprobablities[1][:, 1] * predictedconfusionmatrix_proba[1][0][1]
) + (p_k1_c1 * p_k3_c1)

p_k3_c0 = ci0 * (
    predictedprobablities[2][:, 0] * predictedconfusionmatrix_proba[2][0][0]
    + predictedprobablities[2][:, 1] * predictedconfusionmatrix_proba[2][0][1]
) + (p_k2_c1 * p_k1_c1)



<IPython.core.display.Javascript object>

In [16]:
finalcol = np.zeros((ytest.shape[0], 3))
finalcol[:, 0] = p_k1_c0
finalcol[:, 1] = p_k2_c0
finalcol[:, 2] = p_k3_c0
finalpred = np.argmax(finalcol, axis=1)

ytest = ytest_original.copy()
print(m.accuracy_score(ytest, finalpred))
print(m.confusion_matrix(ytest, finalpred))

0.9814814814814815
[[15  0  0]
 [ 0 26  1]
 [ 0  0 12]]


<IPython.core.display.Javascript object>

In [17]:

finalcol = np.zeros((ytest.shape[0], 3))
finalcol[:, 0] = p_k1_c1
finalcol[:, 1] = p_k2_c1
finalcol[:, 2] = p_k3_c1
finalpred = np.argmin(finalcol, axis=1)

ytest = ytest_original.copy()
print(m.accuracy_score(ytest, finalpred))
print(m.confusion_matrix(ytest, finalpred))


0.9814814814814815
[[15  0  0]
 [ 0 26  1]
 [ 0  0 12]]


<IPython.core.display.Javascript object>